In [88]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
# import seaborn as sns
from scipy import stats
import re
# import scipy.stats as stats

In [89]:
source_file = 'diamonds.csv'
url = 'https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/'
df = pd.read_csv(url + source_file)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [90]:
df_all = pd.get_dummies(df.loc[:,['carat','color', 'cut', 'x', 'y', 'z', 'price']])
df_all.info()
df_all.columns = df_all.columns.str.replace(" ","_")
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   carat          53940 non-null  float64
 1   x              53940 non-null  float64
 2   y              53940 non-null  float64
 3   z              53940 non-null  float64
 4   price          53940 non-null  int64  
 5   color_D        53940 non-null  bool   
 6   color_E        53940 non-null  bool   
 7   color_F        53940 non-null  bool   
 8   color_G        53940 non-null  bool   
 9   color_H        53940 non-null  bool   
 10  color_I        53940 non-null  bool   
 11  color_J        53940 non-null  bool   
 12  cut_Fair       53940 non-null  bool   
 13  cut_Good       53940 non-null  bool   
 14  cut_Ideal      53940 non-null  bool   
 15  cut_Premium    53940 non-null  bool   
 16  cut_Very Good  53940 non-null  bool   
dtypes: bool(12), float64(4), int64(1)
memory usage: 2.

### 2.2 diamond equation #2
Для набора данных `diamonds` рассматриваются несколько линейных регрессий.

In [91]:
formula=(
    f'np.log(price)~'
    f'carat+I(carat**2)+'
    f'color_E + color_F + color_G + color_H + color_I + color_J'
    f'+cut_Good + cut_Very_Good + cut_Premium+cut_Ideal+'
    f'x + y + z'
)
model_d = smf.ols(formula=formula,data=df_all).fit()
model_d.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(price)   R-squared:                       0.949
Model:                            OLS   Adj. R-squared:                  0.949
Method:                 Least Squares   F-statistic:                 6.707e+04
Date:                Sun, 30 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:51:00   Log-Likelihood:                 3006.0
No. Observations:               53940   AIC:                            -5980.
Df Residuals:                   53924   BIC:                            -5838.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 3.9916      0.026    154.426      0.000       3.941       4.042
color_E[T.True]          -0.0302      0.004     -8.359      0.000      -0.037      -0.023
color_F[T.True]          -0.0395      0.004    -10.843      0.000      -0.047      -0.032
color_G[T.True]          -0.0714      0.004    -20.226      0.000      -0.078      -0.064
color_H[T.True]          -0.2101      0.004    -55.418      0.000      -0.218      -0.203
color_I[T.True]          -0.3075      0.004    -72.295      0.000      -0.316      -0.299
color_J[T.True]          -0.4324      0.005    -82.185      0.000      -0.443      -0.422
cut_Good[T.True]          0.1184      0.007     17.875      0.000       0.105       0.131
cut_Very_Good[T.True]     0.1774      0.006     28.831      0.000       0.165       0.189
cut_Premium[T.True]       0.1612      0.006     26.215      0.000       0.149       0.173
cut_Ideal[T.True]         0.2508      0.006     41.516      0.000       0.239       0.263
carat                     2.5355      0.029     88.040      0.000       2.479       2.592
I(carat ** 2)            -0.6458      0.006   -110.559      0.000      -0.657      -0.634
x                         0.3390      0.008     40.767      0.000       0.323       0.355
y                         0.0145      0.004      3.728      0.000       0.007       0.022
z                         0.0631      0.006     10.323      0.000       0.051       0.075
==============================================================================
Omnibus:                    13348.255   Durbin-Watson:                   1.348
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           374928.681
Skew:                           0.582   Prob(JB):                         0.00
Kurtosis:                      15.863   Cond. No.                         361.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [92]:
# Убрали color
formula=(
    f'np.log(price)~'
    f'carat+I(carat**2)'
    f'+cut_Good + cut_Very_Good + cut_Premium+cut_Ideal+'
    f'x + y + z'
)
model_d1 = smf.ols(formula=formula,data=df_all).fit()
model_d1.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(price)   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.936
Method:                 Least Squares   F-statistic:                 8.794e+04
Date:                Sun, 30 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:51:00   Log-Likelihood:                -3097.3
No. Observations:               53940   AIC:                             6215.
Df Residuals:                   53930   BIC:                             6304.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 3.7954      0.029    132.152      0.000       3.739       3.852
cut_Good[T.True]          0.1156      0.007     15.598      0.000       0.101       0.130
cut_Very_Good[T.True]     0.1734      0.007     25.172      0.000       0.160       0.187
cut_Premium[T.True]       0.1553      0.007     22.564      0.000       0.142       0.169
cut_Ideal[T.True]         0.2448      0.007     36.204      0.000       0.232       0.258
carat                     2.3653      0.032     73.523      0.000       2.302       2.428
I(carat ** 2)            -0.6444      0.007    -98.661      0.000      -0.657      -0.632
x                         0.3806      0.009     40.933      0.000       0.362       0.399
y                         0.0163      0.004      3.736      0.000       0.008       0.025
z                         0.0558      0.007      8.143      0.000       0.042       0.069
==============================================================================
Omnibus:                    12690.786   Durbin-Watson:                   1.297
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           230533.103
Skew:                           0.663   Prob(JB):                         0.00
Kurtosis:                      13.041   Cond. No.                         359.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [93]:
# Убрали cut
formula = (
    f'np.log(price)~'
    f'carat+I(carat**2)+'
    f'color_E + color_F + color_G + color_H + color_I + color_J'
    f'+x + y + z'
)

model_d2 = smf.ols(formula=formula,data=df_all).fit()
print(
    summary_col(
        [model_d, model_d1, model_d2],
        float_format='%.3f',
        stars=True
    )
)


                      np.log(price) I np.log(price) II np.log(price) III
------------------------------------------------------------------------
I(carat ** 2)         -0.646***       -0.644***        -0.610***        
                      (0.006)         (0.007)          (0.006)          
Intercept             3.992***        3.795***         4.078***         
                      (0.026)         (0.029)          (0.026)          
R-squared             0.949           0.936            0.946            
R-squared Adj.        0.949           0.936            0.946            
carat                 2.535***        2.365***         2.349***         
                      (0.029)         (0.032)          (0.029)          
color_E[T.True]       -0.030***                        -0.031***        
                      (0.004)                          (0.004)          
color_F[T.True]       -0.039***                        -0.040***        
                      (0.004)                     

#### Уровень значимости 5%.

Тестируется значимость влияния цвета, т.е. гипотеза
 
$H_0:\beta_{colorE}=\beta_{colorF}=\beta_{colorG}=\beta_{colorH}=\beta_{colorI}=\beta_{colorJ}=0$.

In [94]:
hypothesis = (
    f'color_E[T.True] = '
    f'color_F[T.True] = '
    f'color_G[T.True] = '
    f'color_H[T.True] = '
    f'color_I[T.True] = '
    f'color_J[T.True] = 0'
)
f_obs = model_d.f_test(hypothesis).fvalue
print(f'F_obs = {f_obs:3.2f}')

F_obs = 2282.37


In [95]:
rest = model_d1.rsquared
# rest = 0.188
print(f'rest = {rest:3.3f}')
unrest = model_d.rsquared
# unrest = 0.205
print(f'unrest = {unrest:3.3f}')
residual1 = model_d.df_resid
residual2 = model_d1.df_resid
print(f'residual1 = {residual1}')
print(f'residual2 = {residual2}')
j = residual2 - residual1
print(f'J = {j}')

rest = 0.936
unrest = 0.949
residual1 = 53924.0
residual2 = 53930.0
J = 6.0


In [96]:
f_obs = (unrest - rest)/(1-unrest)*residual1/j
print(f'F_obs = {f_obs:.2f}')

F_obs = 2282.37


In [97]:
alpha = 0.05 # Уровень значимости
f_cr = stats.f.ppf(1-alpha,j,len(df_all)-j).round(2)
print(f'F_cr = {f_cr:.2f}')

F_cr = 2.10


In [98]:
prompt = 'Влияние'
reply = 'Значимо' if f_obs > f_cr else 'Незначимо'
print(f'{prompt} {reply}')

Влияние Значимо


Тестируется значимость влияния качества огранки, т.е. гипотеза
 
$H_0:\beta_{cutGood}=\beta_{cutVeryGood}=\beta_{cutPremium}=\beta_{cutIdeal}=0$.

In [99]:
hypothesis = (
    f'cut_Good[T.True] = '
    f'cut_Very_Good[T.True] = '
    f'cut_Premium[T.True] = '
    f'cut_Ideal[T.True] = 0'
)
f_obs = model_d.f_test(hypothesis).fvalue
print(f'F_obs = {f_obs:3.2f}')

F_obs = 797.07


In [100]:
rest = model_d2.rsquared
print(f'rest = {rest:3.3f}')
unrest = model_d.rsquared
print(f'unrest = {unrest:3.3f}')
residual1 = model_d.df_resid
residual2 = model_d2.df_resid
print(f'residual1 = {residual1}')
print(f'residual2 = {residual2}')
j = residual2 - residual1
print(f'J = {j}')

rest = 0.946
unrest = 0.949
residual1 = 53924.0
residual2 = 53928.0
J = 4.0


In [101]:
f_obs = (unrest - rest)/(1-unrest)*residual1/j
print(f'F_obs = {f_obs:.2f}')

F_obs = 797.07


In [102]:
alpha = 0.05 # Уровень значимости
f_cr = stats.f.ppf(1-alpha,j,len(df_all)-j).round(2)
print(f'F_cr = {f_cr:.2f}')

F_cr = 2.37


In [103]:
prompt = 'Влияние'
reply = 'Значимо' if f_obs > f_cr else 'Незначимо'
print(f'{prompt} {reply}')

Влияние Значимо
